<a href="https://colab.research.google.com/github/DESOLATE17/ML/blob/main/%20Lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ансамбли моделей машинного обучения. Часть 1.**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Загрузка данных
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')

# Удаление пропусков
data.dropna(inplace=True)

# Кодирование категориальных признаков
le = LabelEncoder()
for col in data.columns:
    if data[col].dtype == 'object':
        data[col] = le.fit_transform(data[col])

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


### Разделение на признаки и целевую переменную

In [ ]:
X = data.drop('quality', axis=1)
y = data['quality']

In [ ]:
X

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [ ]:
y

0       5
1       5
2       5
3       6
4       5
       ..
1594    5
1595    6
1596    6
1597    5
1598    6
Name: quality, Length: 1599, dtype: int64

### С использованием метода train_test_split разделите выборку на обучающую и тестовую.

In [ ]:

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


###Выбор числовых и категориальных признаков

In [ ]:
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

Создание конвейеров для обработки числовых и категориальных признаков

Numeric Transformer: Для числовых признаков создается конвейер, который сначала заменяет пропущенные значения медианным значением (SimpleImputer(strategy='median')), а затем масштабирует признаки до единичной нормы (StandardScaler()). Это помогает улучшить производительность алгоритмов машинного обучения, делая распределение признаков более гладким и уменьшая влияние масштаба признаков.

Categorical Transformer: Для категориальных признаков создается конвейер, который также заменяет пропущенные значения (SimpleImputer(strategy='constant', fill_value='missing')), но вместо масштабирования применяется кодирование категориальных признаков (OneHotEncoder(handle_unknown='ignore')). Это преобразует категориальные признаки в числовые, создавая новые признаки для каждой уникальной категории, что позволяет алгоритмам машинного обучения обрабатывать эти данные.

In [ ]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

###Создадим конвейер для обработки, так как датасет включает оба типа признаков

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


### Обучение моделей

In [ ]:
models = {
    'Bagging': BaggingClassifier(),
    'RandomForest': RandomForestClassifier(),
    'ExtraRandomTrees': ExtraTreesClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

for name, model in models.items():
    model.fit(preprocessor.fit_transform(X_train), y_train)
    predictions = model.predict(preprocessor.transform(X_test))
    print(f"{name} Accuracy: {accuracy_score(y_test, predictions)}")

Bagging Accuracy: 0.634375
RandomForest Accuracy: 0.671875
ExtraRandomTrees Accuracy: 0.684375
AdaBoost Accuracy: 0.528125
GradientBoosting Accuracy: 0.65
